In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from nb_001b import *
import sys, PIL, matplotlib.pyplot as plt, itertools, math, random, collections, torch
import scipy.stats, scipy.special

from enum import Enum, IntEnum
from torch import tensor, FloatTensor, LongTensor, ByteTensor, DoubleTensor, HalfTensor, ShortTensor
from operator import itemgetter, attrgetter
from numpy import cos, sin, tan, tanh, log, exp

from functools import reduce
from collections import defaultdict, abc, namedtuple, Iterable
from PIL import Image

# CIFAR subset data

First we want to view our data to check if everything is how we expect it to be.

## Setup

In [ ]:
DATA_PATH = Path('data')
PATH = DATA_PATH/'cifar10_dog_air'
TRAIN_PATH = PATH/'train'

In [ ]:
dog_fn = list((TRAIN_PATH/'dog').iterdir())[0]
dog_image = Image.open(dog_fn)
dog_image.resize((256,256))

In [ ]:
air_fn = list((TRAIN_PATH/'airplane').iterdir())[0]
air_image = Image.open(air_fn)
air_image.resize((256,256))

## Simple Dataset/Dataloader

We will build a Dataset class for our image files. A Dataset class needs to have two functions: length and get-item. Our FilesDataset additionally gets the image files from their respective directories and transforms them to tensors.

In [ ]:
#export
def find_classes(folder):
    classes = [d for d in folder.iterdir()
               if d.is_dir() and not d.name.startswith('.')]
    assert(len(classes)>0)
    return sorted(classes, key=lambda d: d.name)

def get_image_files(c):
    return [o for o in list(c.iterdir())
            if not o.name.startswith('.') and not o.is_dir()]

def pil2tensor(image):
    arr = torch.ByteTensor(torch.ByteStorage.from_buffer(image.tobytes()))
    arr = arr.view(image.size[1], image.size[0], -1)
    arr = arr.permute(2,0,1)
    return arr.float().div_(255)

def open_image(fn):
    x = PIL.Image.open(fn).convert('RGB')
    return pil2tensor(x)

In [ ]:
#export
class FilesDataset(Dataset):
    def __init__(self, folder, classes=None):
        self.fns, self.y = [], []
        if classes is None: classes = [cls.name for cls in find_classes(folder)]
        self.classes = classes
        for i, cls in enumerate(classes):
            fnames = get_image_files(folder/cls)
            self.fns += fnames
            self.y += [i] * len(fnames)
        
    def __len__(self): return len(self.fns)
    def __getitem__(self,i): return open_image(self.fns[i]),self.y[i]

In [ ]:
train_ds = FilesDataset(PATH/'train')
valid_ds = FilesDataset(PATH/'test')

In [ ]:
len(train_ds), len(valid_ds)

In [ ]:
#export
def image2np(image): return image.cpu().permute(1,2,0).numpy()

In [ ]:
x,y = train_ds[0]
plt.imshow(image2np(x))
print(train_ds.classes[y])

In [ ]:
bs=64

In [ ]:
data = DataBunch(train_ds, valid_ds, bs=bs)
len(data.train_dl), len(data.valid_dl)

In [ ]:
#export
def show_image(img, ax=None, figsize=(3,3), hide_axis=True):
    if ax is None: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(image2np(img))
    if hide_axis: ax.axis('off')

def show_image_batch(dl, classes, rows=None, figsize=(12,15)):
    x,y = next(iter(dl))
    if rows is None: rows = int(math.sqrt(len(x)))
    show_images(x[:rows*rows],y[:rows*rows],rows, classes)

def show_images(x,y,rows, classes, figsize=(9,9)):
    fig, axs = plt.subplots(rows,rows,figsize=figsize)
    for i, ax in enumerate(axs.flatten()):
        show_image(x[i], ax)
        ax.set_title(classes[y[i]])
    plt.tight_layout()

In [ ]:
show_image_batch(data.train_dl, train_ds.classes, 6)

# Data augmentation

We are going augment our data to increase our training set with artificial images. These new images are basically "free" data that we can use in our training to help our model generalize better (reduce overfitting).

## Lighting

We will start by changing the **brightness** and **contrast** of our images.

### Method

**Brightness**

Brightness refers to where does our image stand on the dark-light spectrum. Brightness is applied by adding a positive constant to each of the image's channels. This works because each of the channels in an image goes from 0 (darkest) to 255 (brightest) in a dark-light continum. (0, 0, 0) is black (total abscence of light) and (255, 255, 255) is white (pure light). You can check how this works by experimenting by yourself [here](https://www.w3schools.com/colors/colors_rgb.asp).

_Parameters_

1. **Change** How much brightness do we want to add to (or take from) the image.

    Domain: Real numbers
    
**Contrast**

Contrast refers to how sharp a distinction there is between birghter and darker sections of our image. To increase contrast we need darker pixels to be darker and lighter pixels to be lighter. In other words, we would like channels with a value smaller than 128 to decrease and channels with a value of greater than 128 to increase.

_Parameters_

1. **Scale** How much contrast do we want to add to (or remove from) the image.

    Domain: [0, +inf]
    
***On logit and sigmoid***

Notice that for both transformations we first apply the logit to our tensor, then apply the transformation and finally take the sigmoid. This is important for two reasons. 

First, we don't want to overflow our tensor values. In other words, we need our final tensor values to be between [0,1].  Imagine, for instance, a tensor value at 0.99. We want to increase its brightness, but we can’t go over 1.0. By doing logit first, which first moves our space to -inf to +inf, this works fine. The same applies to contrast if we have a scale S > 1 (might make some of our tensor values greater than one).

Second, when we apply contrast, we need to affect the dispersion of values around the middle value. Say we want to increase contrast. Then we need the bright values (>0.5) to get brighter and dark values (<0.5) to get darker. We must first transform our tensor values so our values which were originally <0.5 are now negative and our values which were originally >0.5 are positive. This way, when we multiply by a constant, the dispersion around 0 will increase. The logit function does exactly this and allows us to increase or decrease dispersion around a mid value.

### Implementation

In [ ]:
#export
def logit(x):  return -(1/x-1).log()
def logit_(x): return (x.reciprocal_().sub_(1)).log_().neg_()

def brightness(x, change): return x.add_(scipy.special.logit(change))
def contrast(x, scale): return x.mul_(scale)

def _apply_lighting(x, func):
    if func is None: return x
    return func(logit_(x)).sigmoid()

def apply_lighting(func): return partial(_apply_lighting, func=func)

In [ ]:
def apply_contrast(scale): return apply_lighting(partial(contrast, scale=scale))

In [ ]:
x = lambda: train_ds[1][0]

In [ ]:
_,axes = plt.subplots(1,4, figsize=(12,3))

show_image(x(), axes[0])
show_image(apply_contrast(1.0)(x()), axes[1])
show_image(apply_contrast(0.5)(x()), axes[2])
show_image(apply_contrast(2.0)(x()), axes[3])

In [ ]:
def apply_brightness(change):
    return apply_lighting(partial(brightness, change=change))

In [ ]:
_,axes = plt.subplots(1,4, figsize=(12,3))

show_image(x(), axes[0])
show_image(apply_brightness(0.5)(x()), axes[1])
show_image(apply_brightness(0.8)(x()), axes[2])
show_image(apply_brightness(0.2)(x()), axes[3])

In [ ]:
#export
def listify(p=None, q=None):
    if p is None: p=[]
    elif not isinstance(p, Iterable): p=[p]
    n = q if type(q)==int else 1 if q is None else len(q)
    if len(p)==1: p = p * n
    return p

def compose(funcs):
    def _inner(x, *args, **kwargs):
        for f in listify(funcs): x = f(x, *args, **kwargs)
        return x
    return _inner if funcs else None

In [ ]:
def apply_brightness_contrast(scale_contrast, change_brightness):
    return apply_lighting(compose([
        partial(contrast, scale=scale_contrast),
        partial(brightness, change=change_brightness)
    ]))

In [ ]:
_,axes = plt.subplots(1,4, figsize=(12,3))

show_image(apply_brightness_contrast(0.75, 0.7)(x()), axes[0])
show_image(apply_brightness_contrast(1.3,  0.3)(x()), axes[1])
show_image(apply_brightness_contrast(1.3,  0.7)(x()), axes[2])
show_image(apply_brightness_contrast(0.75, 0.3)(x()), axes[3])

## Random lighting

Next, we will make our previous transforms random since we are interested in automatizing the pipeline. We will achieve this by making our parameters stochastic with a specific distribution. 

We will use a <a href="https://en.wikipedia.org/wiki/Uniform_distribution_(continuous)"> uniform</a> distribution for brightness change since its domain is the real numbers and the impact varies linearly with the scale. For contrast we use [log_uniform](https://www.vosesoftware.com/riskwiki/LogUniformdistribution.php) for two reasons. First, contrast scale has a domain of [0, inf]. Second, the impact of the scale in the transformation is non-linear (i.e. 0.5 is as extreme as 2.0, 0.2 is as extreme as 5). The log_uniform function is appropriate because it has the same domain and correctly represents the non-linearity of the transform, P(0.5) = P(2).

In [ ]:
#export
def uniform(low, high, size=None):
    return random.uniform(low,high) if size is None else torch.FloatTensor(size).uniform_(low,high)

def log_uniform(low, high, size=None):
    res = uniform(log(low), log(high), size)
    return exp(res) if size is None else res.exp_()

def rand_bool(p, size=None): return uniform(0,1,size)<p

In [ ]:
scipy.stats.gmean([log_uniform(0.5,2.0) for _ in range(1000)])

In [ ]:
# TfmType = IntEnum('TfmType', 'Start Affine Coord Pixel Lighting')

In [ ]:
#export
import inspect
from copy import copy,deepcopy

def get_default_args(func):
    return {k: v.default
            for k, v in inspect.signature(func).parameters.items()
            if v.default is not inspect.Parameter.empty}

class Transform():
    def __init__(self, func, p=1., **kwargs):
        self.func,self.p,self.kw = func,p,kwargs
        self.def_args = get_default_args(func)
        self.params = copy(func.__annotations__)
        self.params.pop('return')

    @property
    def name(self): return self.__class__.__name__
    
    def __repr__(self):
        return f'{self.func.__name__}_tfm->{self.name}; {self.kw} (p={self.p})'

    def resolve(self):
        self.resolved = {}
        for k,v in self.params.items():
            if k not in self.kw and k in self.def_args: self.resolved[k] = self.def_args[k]
            else: self.resolved[k] = v(*listify(self.kw[k]))
        self.do_run = rand_bool(self.p)

    def __call__(self, x, *args, **kwargs):
        return self.func(x, *args, **self.resolved, **kwargs) if self.do_run else x
    
    def apply(self, tfms=None):
        if tfms is None: tfms=self
        return compose(tfms)

class TfmLighting(Transform):
    def apply(self, tfms=None): return apply_lighting(super().apply(tfms))

In [ ]:
def brightness(x, change:uniform) -> TfmLighting:
    return x.add_(scipy.special.logit(change))

def contrast(x, scale:log_uniform) -> TfmLighting:
    return x.mul_(scale)

In [ ]:
brightness.__annotations__

In [ ]:
contrast_tfm = partial(contrast.__annotations__['return'], contrast)
tfm = contrast_tfm(scale=(0.3,3))
tfm.resolve()
tfm,tfm.resolved

In [ ]:
# all the same
tfm.resolve()

_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes: show_image(tfm.apply()(x()), ax)

In [ ]:
tfm = contrast_tfm(scale=(0.3,3), p=0.5)

# different
_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes:
    tfm.resolve()
    show_image(apply_lighting(tfm)(x()), ax)

## Decorator and composition

We are interested in composing the transform functions so as to apply them all at once. We will try to feed a list of transforms to our pipeline for it to apply all of them.

Applying a function to our transforms before calling them in Python is easiest if we use a decorator. You can find more about decorators [here](https://www.thecodeship.com/patterns/guide-to-python-function-decorators/).

In [ ]:
#export
def reg_transform(func):
    tfm = partial(func.__annotations__['return'], func)
    setattr(sys.modules[func.__module__], f'{func.__name__}_tfm', tfm)
    return func

@reg_transform
def brightness(x, change:uniform) ->TfmLighting: return x.add_(scipy.special.logit(change))

@reg_transform
def contrast(x, scale:log_uniform)->TfmLighting: return x.mul_(scale)

def resolve_tfms(tfms):
    for f in listify(tfms): f.resolve()

In [ ]:
def _apply_tfms(tfms, x):
    if not tfms: return x
    resolve_tfms(tfms)
    return tfms[0].apply(tfms)(x.clone())

def apply_tfms(tfms): return partial(_apply_tfms, tfms)

In [ ]:
x = train_ds[1][0]

In [ ]:
tfms = [contrast_tfm(scale=(0.3,3.0), p=0.75),
        brightness_tfm(change=(0.35,0.65), p=0.75)]

_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes: show_image(apply_tfms(tfms)(x), ax)

In [ ]:
_,axes = plt.subplots(2,4, figsize=(12,6))
for i in range(4):
    tfm = apply_tfms(tfms)
    show_image(tfm(x), axes[0][i])
    show_image(tfm(x), axes[1][i])

In [ ]:
show_image(apply_tfms([])(x))

# Affine

We will now add affine transforms that operate on the coordinates instead of pixels like the lighting transforms we just saw. An [affine transformation](https://en.wikipedia.org/wiki/Affine_transformation) is a function "(...) between affine spaces which preserves points, straight lines and planes." 

## Details

Our implementation first creates a grid of coordinates for the original image. The grid is normalized to a [-1, 1] range with (-1, -1) representing the top left corner, (1, 1) the bottom right corner and (0, 0) the center. Next, we build an affine matrix representing our desired transform and we multiply it by our original grid coordinates. The result will be a set of x, y coordinates which references where in the input image will each of the pixels in the output image be mapped. It has a size of w \* h \* 2 since it needs two coordinates for each of the h * w pixels of the output image. 

This is clearest if we see it graphically. We will build an affine matrix of the following form:

`[[a, b, e],
 [c, d, f]]`


with which we will transform each pair of x, y coordinates in our original grid into our transformation grid:


`[[a, b],       [[x],       [[e],       [[x'],
  [c, d]]   x    [y]]    +   [f]]    =   [y']]`   

So after the transform we will get a new grid with which to map our input image into our output image. This will be our **map of where from exactly does our transformation source each pixel in the output image**.

**Enter problems**

Affine transforms face two problems that must be solved independently:
1. **The interpolation problem**: The result of our transformation gives us float coordinates, and we need to decide, for each (i,j), how to assign these coordinates to pixels in the input image.
2. **The missing pixel problem**: The result of our transformation may have coordinates which exceed the [-1, 1] range of our original grid and thus fall outside of our original grid.

**Solutions to problems**

1.  **The interpolation problem**: We will perform a [bilinear interpolation](https://en.wikipedia.org/wiki/Bilinear_interpolation). This takes an average of the values of the pixels corresponding to the four points in the grid surrounding the result of our transformation, with weights depending on how close we are to each of those points. 
2. **The missing pixel problem**: For these values we need padding, and we face a few options:

    1. Adding zeros on the side (so the pixels that fall out will be black)
    2. Replacing them by the value at the border
    3. Mirroring the content of the picture on the other side (reflect padding).
    
    
### Transformation Method

**Zoom**

Zoom changes the focus of the image according to a scale. If a scale of >1 is applied, grid pixels will be mapped to coordinates that are more central than the pixel's coordinates (closer to 0,0) while if a scale of <1 is applied, grid pixels will be mapped to more perispheric coordinates (closer to the borders) in the input image.

We can also translate our transform to zoom into a non-centrical area of the image. For this we use $col_c$ which displaces the x axis and $row_c$ which displaces the y axis.

_Parameters_

1. **Scale** How much do we want to zoom in or out to our image.

    Domain:  Real numbers
        
2. **Col_pct** How much do we want to displace our zoom along the x axis.

    Domain: Real numbers between 0 and 1
    
    
3. **Row_pct** How much do we want to displace our zoom along the y axis.

    Domain: Real numbers between 0 and 1
    

<u>Affine matrix</u>

`[[1/scale, 0,       col_c],
  [0,       1/scale, row_c]]`


**Rotate**

Rotate shifts the image around its center in a given angle theta. The rotation is counterclockwise if theta is positive and clockwise if theta is negative. If you are curious about the derivation of the rotation matrix you can find it [here](https://matthew-brett.github.io/teaching/rotation_2d.html).

_Parameters_

1. **Degrees** By which angle do we want to rotate our image.

    Domain: Real numbers
        
<u>Affine matrix</u>

`[[cos(theta), -sin(theta), 0],
  [sin(theta),  cos(theta), 0]]`

## Deterministic affine

In [ ]:
#export
def grid_sample_nearest(input, coords, padding_mode='zeros'):
    if padding_mode=='border': coords.clamp(-1,1)
    bs,ch,h,w = input.size()
    sz = torch.tensor([w,h]).float()[None,None]
    coords.add_(1).mul_(sz/2)
    coords = coords[0].round_().long()
    if padding_mode=='zeros':
        mask = (coords[...,0] < 0) + (coords[...,1] < 0) + (coords[...,0] >= w) + (coords[...,1] >= h)
        mask.clamp_(0,1)
    coords[...,0].clamp_(0,w-1)
    coords[...,1].clamp_(0,h-1)
    result = input[...,coords[...,1],coords[...,0]]
    if padding_mode=='zeros': result[...,mask] = result[...,mask].zero_()
    return result

In [ ]:
#export
def grid_sample(x, coords, mode='bilinear', padding_mode='reflect'):
    if padding_mode=='reflect': padding_mode='reflection'
    if mode=='nearest': return grid_sample_nearest(x[None], coords, padding_mode)[0]
    return F.grid_sample(x[None], coords, mode=mode, padding_mode=padding_mode)[0]

def affine_grid(x, matrix, size=None):
    if size is None: size = x.size()
    elif isinstance(size, int): size=(x.size(0), size, size)
    return F.affine_grid(matrix[None,:2], torch.Size((1,)+size))

In [ ]:
def rotate(degrees):
    angle = degrees * math.pi / 180
    return [[cos(angle), -sin(angle), 0.],
            [sin(angle),  cos(angle), 0.],
            [0.        ,  0.        , 1.]]

In [ ]:
m = rotate(30)
m = x.new_tensor(m)
c = affine_grid(x, m)
img2 = grid_sample(x, c, padding_mode='zeros')
show_image(img2)

In [ ]:
#export
def affines_mat(img, matrices=None):
    matrices = [img.new_tensor(m) for m in matrices if m is not None]
    if not matrices: return None
    return reduce(torch.matmul, matrices, torch.eye(3))

def affine_mult(c,m):
    if m is None: return c
    size = c.size()
    c = c.view(-1,2)
    c = torch.addmm(m[:2,2], c,  m[:2,:2].t()) 
    return c.view(size)

def _apply_affine(c, size=None, mats=None, func=None, **kwargs):
    return affine_mult(c, affines_mat(c, mats))

def apply_affine(mats): return partial(_apply_affine, mats=mats)

def affine_only(x, mats, size=None, **kwargs):
    tfm = apply_affine(mats)
    c = affine_grid(x, torch.eye(3), size=size)
    c = tfm(c)
    return grid_sample(x, c, **kwargs)

In [ ]:
img2 = affine_only(x, [rotate(30)])
show_image(img2)

In [ ]:
def zoom(scale: uniform, row_pct=0.5, col_pct=0.5):
    s = 1-1/scale
    col_c = s * (2*col_pct - 1)
    row_c = s * (2*row_pct - 1)
    return [[1/scale, 0,       col_c],
            [0,       1/scale, row_c],
            [0,       0,       1.    ]]

In [ ]:
show_image(affine_only(x, [zoom(0.6)]))

In [ ]:
show_image(affine_only(x, [zoom(0.6)], padding_mode='zeros'))

In [ ]:
show_image(affine_only(x, [zoom(2, 0.2, 0.2)]))

In [ ]:
img2 = affine_only(x,    [rotate(30)])
img2 = affine_only(img2, [zoom(1.6)])
show_image(img2)

In [ ]:
show_image(affine_only(x, [zoom(1.6), rotate(30)]))

In [ ]:
show_image(x)

In [ ]:
m = [zoom(1.6, 0.8, 0.2), rotate(30)]
show_image(affine_only(x, m, size=48))

In [ ]:
m = [zoom(1.6, 0.8, 0.2), rotate(30)]
show_image(affine_only(x, m, size=24), hide_axis=False)

In [ ]:
m = [zoom(1.6, 0.8, 0.2), rotate(30)]
show_image(affine_only(x, m, size=48, mode='nearest'))

In [ ]:
show_image(affine_only(x, [zoom(1.6)]))

In [ ]:
show_image(affine_only(x, []))

## Random affine

As we did with the Lighting transform, we now want to build randomness into our pipeline so we can increase the automatization of the transform process. 

We will use a uniform distribution for both our transforms since their impact is linear and their domain is the real numbers.

**Apply all transforms**

We will make all transforms try to do as little calculations as possible.

We do only one affine transformation by multiplying all the affine matrices of the transforms, then we apply to the coords any non-affine transformation we might want (jitter, elastic distorsion). Next, we crop the coordinates we want to keep and, by doing it before the interpolation, we don't need to compute pixel values that won't be used afterwards. Finally we perform the interpolation and we apply all the transforms that operate pixelwise (brightness, contrast).

In [ ]:
#export
class TfmAffine(Transform):
    def __call__(self, *args, **kwargs):
        return self.func(*args, **self.resolved, **kwargs) if self.do_run else None
    
    def apply(self, tfms=None): return apply_affine([o() for o in tfms])

class TfmCoord(Transform): pass
class TfmStart(Transform): pass
class TfmPixel(Transform): pass

def _apply_tfms1(tfms, x, **kwargs):
    return tfms[0].apply(tfms)(x, **kwargs) if tfms else x

class Transforms():
    def __init__(self, *tfms:Collection[Transform]):
        self.tfms = tfms
        self.grouped = {o: [t for t in tfms if isinstance(t,o)]
                        for o in tfm_types}

    def resolve(self): resolve_tfms(self.tfms)

    def apply_group(self, k, x, **kwargs):
        return _apply_tfms1(self.grouped[k], x, **kwargs)

    def apply(self, x, size=None, **kwargs):
        if not self.tfms: return x
        x = x.clone()
        x = self.apply_group(TfmStart, x)
        if self.grouped[TfmAffine] or self.grouped[TfmCoord] or (size is not None):
            c = affine_grid(x, torch.eye(3), size=size)
            c = self.apply_group(TfmCoord, c, size=x.size())
            c = self.apply_group(TfmAffine, c)
            x = grid_sample(x, c, **kwargs)
        x = self.apply_group(TfmLighting, x)
        x = self.apply_group(TfmPixel, x)
        return x

In [ ]:
tfm_types = [TfmStart, TfmAffine, TfmCoord, TfmLighting, TfmPixel]

In [ ]:
#export
@reg_transform
def rotate(degrees:uniform) -> TfmAffine:
    angle = degrees * math.pi / 180
    return [[cos(angle), -sin(angle), 0.],
            [sin(angle),  cos(angle), 0.],
            [0.        ,  0.        , 1.]]

def get_zoom_mat(sw, sh, c, r):
    return [[sw, 0,  c],
            [0, sh,  r],
            [0,  0, 1.]]

@reg_transform
def zoom(scale:uniform=1.0, row_pct:uniform=0.5, col_pct:uniform=0.5) -> TfmAffine:
    s = 1-1/scale
    col_c = s * (2*col_pct - 1)
    row_c = s * (2*row_pct - 1)
    return get_zoom_mat(1/scale, 1/scale, col_c, row_c)

@reg_transform
def squish(scale:uniform=1.0, row_pct:uniform=0.5, col_pct:uniform=0.5) -> TfmAffine:
    if scale <= 1: 
        col_c = (1-scale) * (2*col_pct - 1)
        return get_zoom_mat(scale, 1, col_c, 0.)
    else:          
        row_c = (1-1/scale) * (2*row_pct - 1)
        return get_zoom_mat(1, 1/scale, 0., row_c)

In [ ]:
tfms = Transforms(rotate_tfm(degrees=(-45,45.), p=0.75),
        zoom_tfm(scale=(0.5,2.0), p=0.75))

_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes:
    tfms.resolve()
    show_image(tfms.apply(x), ax)

In [ ]:
tfms = Transforms(rotate_tfm(degrees=(-45,45.), p=0.75),
        zoom_tfm(scale=(1.0,2.0), row_pct=(0,1.), col_pct=(0,1.)))

_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes:
    tfms.resolve()
    show_image(tfms.apply(x, size=64, padding_mode='zeros'), ax)

In [ ]:
scales = [0.75,0.9,1.1,1.33]

_,axes = plt.subplots(1,4, figsize=(12,3))
for i, ax in enumerate(axes): 
    show_image(affine_only(x, [squish(scales[i])], size=64, padding_mode='zeros'), ax)

In [ ]:
tfms = Transforms(squish_tfm(scale=(0.5,2), row_pct=(0,1.), col_pct=(0,1.)))

_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes:
    tfms.resolve()
    show_image(tfms.apply(x, size=64, padding_mode='zeros'), ax)

# Coord and pixel

## Jitter / flip

The last two transforms we will use are **jitter** and **flip**. 

**Jitter**

Jitter is a transform which adds a random value to each of the pixels to make them somewhat different than the original ones. In our implementation we first get a random number between (-1, 1) and we multiply it by a constant M which scales it.

_Parameters_

1. **Magnitude** How much random noise do we want to add to each of the pixels in our image.

    Domain: Real numbers between 0 and 1.
    
**Flip**

Flip is a transform that reflects the image on a given axis.

_Parameters_

1. **P** Probability of applying the transformation to an input.

    Domain: Real numbers between 0 and 1.

In [ ]:
#export
@reg_transform
def jitter(x, size, magnitude: uniform) -> TfmCoord:
    return x.add_((torch.rand_like(x)-0.5)*magnitude*2)

@reg_transform
def flip_lr(x) -> TfmPixel: return x.flip(2)

In [ ]:
tfms = Transforms(jitter_tfm(magnitude=(0,0.1)))

_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes:
    tfms.resolve()
    show_image(tfms.apply(x), ax)

In [ ]:
tfms = Transforms(flip_lr_tfm(p=0.5))

_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes:
    tfms.resolve()
    show_image(tfms.apply(x), ax)

## Combine

In [ ]:
tfms = Transforms(
    flip_lr_tfm(p=0.5),
    rotate_tfm(degrees=(-45,45.), p=0.5),
    zoom_tfm(scale=(0.6,1.6), p=0.8),
    contrast_tfm(scale=(0.5,2.0)),
    brightness_tfm(change=(0.3,0.7))
)

In [ ]:
_,axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes:
    tfms.resolve()
    show_image(tfms.apply(x), ax)

In [ ]:
_,axes = plt.subplots(2,4, figsize=(12,6))
for i in range(4):
    tfms.resolve()
    show_image(tfms.apply(x, padding_mode='zeros', size=48), axes[0][i])
    show_image(tfms.apply(x, mode='nearest'), axes[1][i])

## RandomResizedCrop (Torchvision version)

In [ ]:
#export
def compute_zs_mat(sz, scale, squish, invert, row_pct, col_pct):
    orig_ratio = math.sqrt(sz[2]/sz[1])
    for s,r, i in zip(scale,squish, invert):
        s,r = math.sqrt(s),math.sqrt(r)
        if s * r <= 1 and s / r <= 1: #Test if we are completely inside the picture
            w,h = (s/r, s*r) if i else (s*r,s/r)
            w /= orig_ratio
            h *= orig_ratio
            col_c = (1-w) * (2*col_pct - 1)
            row_c = (1-h) * (2*row_pct - 1)
            return get_zoom_mat(w, h, col_c, row_c)
        
    #Fallback, hack to emulate a center crop without cropping anything yet.
    if orig_ratio > 1: return get_zoom_mat(1/orig_ratio**2, 1, 0, 0.)
    else:              return get_zoom_mat(1, orig_ratio**2, 0, 0.)

@reg_transform
def zoom_squish(c, size, scale: uniform = 1.0, squish: uniform=1.0, invert: rand_bool = False, 
                row_pct:uniform = 0.5, col_pct:uniform = 0.5) -> TfmCoord:
    #This is intended for scale, squish and invert to be of size 10 (or whatever) so that the transform
    #can try a few zoom/squishes before falling back to center crop (like torchvision.RandomResizedCrop)
    m = compute_zs_mat(size, scale, squish, invert, row_pct, col_pct)
    return affine_mult(c, FloatTensor(m))

In [ ]:
random_resized_crop = zoom_squish_tfm(scale=(0.5,1,10), squish=(0.75,1.33,10), invert=(0.5,10),
                                      row_pct=(0,1.), col_pct=(0,1.))
tfms = Transforms(random_resized_crop)

In [ ]:
_,axes = plt.subplots(2,4, figsize=(12,6))
for i in range(4):
    tfms.resolve()
    show_image(tfms.apply(x, size=(3,48,48)), axes[0][i])
    show_image(tfms.apply(x, mode='nearest', size=(3,32,32)), axes[1][i])